In [2]:
!pip install stable_baselines3

/Users/nicholaskhorasani/Documents/Columbia_University/Spring2025/ML and Climate/EV2Gym-DrivewayV1/venv/bin/pip: line 2: /Users/nicholaskhorasani/Documents/Columbia_University/Spring2025/ML and Climate/EV2Gym-Driveways/venv/bin/python3: No such file or directory
/Users/nicholaskhorasani/Documents/Columbia_University/Spring2025/ML and Climate/EV2Gym-DrivewayV1/venv/bin/pip: line 2: exec: /Users/nicholaskhorasani/Documents/Columbia_University/Spring2025/ML and Climate/EV2Gym-Driveways/venv/bin/python3: cannot execute: No such file or directory


In [3]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from ev2gym_driveway.ev import EV2GymDriveway
from ev2gym_driveway.rl_agent.state import state_function_basic_profit_view
from ev2gym_driveway.rl_agent.reward import reward_function_profit_only

def make_env():
    return EV2GymDriveway(
        config_file="configs/your_config.yaml",
        state_function=state_function_basic_profit_view,
        reward_function=reward_function_profit_only,
        verbose=False  # change to True if you want logs from sim
    )

# Wrap in DummyVecEnv for SB3
env = DummyVecEnv([make_env])

# Create and train PPO agent
model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    tensorboard_log="./ev2gym_logs/",  # optional
    n_steps=2048,
    batch_size=64,
    learning_rate=3e-4
)

model.learn(total_timesteps=100_000)

# Save the model
model.save("ppo_ev2gym_agent")


ModuleNotFoundError: No module named 'stable_baselines3'

In [ ]:
from stable_baselines3 import PPO
from ev2gym_driveway.EV2GymDriveway import EV2GymDriveway
from ev2gym_driveway.rl_agent.state import state_function_basic_profit_view
from ev2gym_driveway.rl_agent.reward import reward_function_profit_only

env = EV2GymDriveway(
    config_file="configs/your_config.yaml",
    state_function=state_function_basic_profit_view,
    reward_function=reward_function_profit_only,
    verbose=True
)

model = PPO.load("ppo_ev2gym_agent")

obs, _ = env.reset()
done = False
total_reward = 0

while not done:
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, _, info = env.step(action)
    total_reward += reward

print(f"Total reward: {total_reward:.2f}")
print(f"Profit: {info['total_money_earned'] - info['total_money_spent']:.2f}")
